In [13]:
import pandas as pd
import numpy as np

In [14]:
cna_df = pd.read_csv('../data/feature_eng/all_cancers_cna_df.csv').drop("Unnamed: 0", axis = 1)
tsv_df = pd.read_csv('../data/feature_eng/all_cancers_tsv_df.csv').drop("Unnamed: 0", axis = 1)
vcf_df = pd.read_csv('../data/feature_eng/all_cancers_vcf_df.csv').drop("Unnamed: 0", axis = 1)

snv_features = pd.read_csv('../data/feature_eng/snv_features.csv').drop("Unnamed: 0", axis = 1)
cna_features = pd.read_csv('../data/feature_eng/cna_features.csv').drop("Unnamed: 0", axis = 1)
sv_features = pd.read_csv('../data/feature_eng/sv_features.csv').drop("Unnamed: 0", axis = 1)

In [15]:
# one label per sample_id
labels = (
    vcf_df[["sample_id", "cancer_type"]]
    .drop_duplicates()
    .reset_index(drop=True)
)

labels

,sample_id,cancer_type
0,sim3_Lymph-CLL,Lymph-CLL
1,sim54_Lymph-CLL,Lymph-CLL
2,sim40_Lymph-CLL,Lymph-CLL
3,sim68_Lymph-CLL,Lymph-CLL
4,sim97_Lymph-CLL,Lymph-CLL
...,...,...
795,sim49_Breast-AdenoCa,Breast-AdenoCa
796,sim48_Breast-AdenoCa,Breast-AdenoCa
797,sim60_Breast-AdenoCa,Breast-AdenoCa
798,sim74_Breast-AdenoCa,Breast-AdenoCa


In [16]:
# SNVs (VCFs) only
baseline_features = (
    snv_features
    .merge(labels, on="sample_id", how="inner")
)
# CNAs + SVs
multi_omic = (
    snv_features
    .merge(cna_features,on="sample_id", how="left")
    .merge(sv_features,on="sample_id", how="left")
    .merge(labels,on="sample_id", how="left")
)
multi_omic.drop("cancer_type", axis = 1) # cannot have target data as a field

target_col = "cancer_type"

X_base = baseline_features.drop(columns=["sample_id", target_col])
y_base = baseline_features[target_col]

X_multi = multi_omic.drop(columns=["sample_id", target_col])
y_multi = multi_omic[target_col]

print("Baseline X/y:", X_base.shape, len(y_base))
print("Multi-omic X/y:", X_multi.shape, len(y_multi))


Baseline X/y: (800, 224) 800
Multi-omic X/y: (800, 235) 800


In [22]:
print(X_base.columns) # features
print(y_base) # true labels

Index(['snv_total', 'snv_chr_1', 'snv_chr_10', 'snv_chr_11', 'snv_chr_12',
       'snv_chr_13', 'snv_chr_14', 'snv_chr_15', 'snv_chr_16', 'snv_chr_17',
       ...
       'snv_ms_driver_TRIO_Intron', 'snv_ms_driver_TSC1_Intron',
       'snv_ms_driver_TSC2_Intron', 'snv_ms_driver_VHL_Coding',
       'snv_ms_driver_WHSC1L1_Intron', 'snv_ms_driver_WT1_Intron',
       'snv_ms_driver_WWOX_Intron', 'snv_ms_driver_ZC3H11A_Intron',
       'snv_ms_driver_ZFHX3_Intron', 'snv_ms_driver_ZNF292_Coding'],
      dtype='object', length=224)
0      Breast-AdenoCa
1       CNS-PiloAstro
2         Eso-AdenoCa
3          Kidney-RCC
4           Liver-HCC
            ...      
795        Kidney-RCC
796         Liver-HCC
797         Lymph-CLL
798    Panc-Endocrine
799     Prost-AdenoCA
Name: cancer_type, Length: 800, dtype: object


In [18]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
import pandas as pd

seed = 456

Xb_train, Xb_val, yb_train, yb_val = train_test_split(X_base, y_base, test_size=0.2, random_state=seed, stratify=y_base)
Xm_train, Xm_val, ym_train, ym_val = train_test_split(X_multi, y_multi, test_size=0.2, random_state=seed, stratify=y_multi)

scaler_base = StandardScaler()
Xb_train_scaled = scaler_base.fit_transform(Xb_train)
Xb_val_scaled   = scaler_base.transform(Xb_val)

scaler_multi = StandardScaler()
Xm_train_scaled = scaler_multi.fit_transform(Xm_train)
Xm_val_scaled = scaler_multi.transform(Xm_val)

# Modeling
logreg_base = LogisticRegression(
    multi_class = "multinomial",
    solver = "lbfgs",
    max_iter = 1000, #250, 300, 500
    n_jobs = -1,
    random_state = seed
)

logreg_multi = LogisticRegression(
    multi_class = "multinomial",
    solver = "lbfgs",
    max_iter = 1000,
    n_jobs = -1,
    random_state = seed
)

rf_multi = RandomForestClassifier(
    n_estimators = 300,
    max_depth = None,
    n_jobs = -1,
    random_state = seed
)

#train and evaluate

# Fitting
logreg_base.fit(Xb_train_scaled, yb_train)
logreg_multi.fit(Xm_train_scaled, ym_train)
rf_multi.fit(Xm_train_scaled, ym_train)

# Testing
yb_pred = logreg_base.predict(Xb_val_scaled)
ym_pred_lr = logreg_multi.predict(Xm_val_scaled)
ym_pred_rf = rf_multi.predict(Xm_val_scaled)

# Calculate accuracy
base_acc = accuracy_score(yb_val, yb_pred)
multi_lr_acc = accuracy_score(ym_val, ym_pred_lr)
multi_rf_acc = accuracy_score(ym_val, ym_pred_rf)

# Calculate F1s
base_f1 = f1_score(yb_val, yb_pred, average="macro") #
multi_lr_f1 = f1_score(ym_val, ym_pred_lr, average="macro")
multi_rf_f1 = f1_score(ym_val, ym_pred_rf, average="macro") # Multi-Omic + Random Forest

# nice dataframe for displaying results
results = pd.DataFrame([
    {"Features": "SNV-only",
    "Model": "LogisticRegression",
    "Accuracy": base_acc,
    "Macro-F1": base_f1,},

    {"Features": "SNV + CNA + SV",
    "Model": "LogisticRegression",
    "Accuracy": multi_lr_acc,
    "Macro-F1": multi_lr_f1,},
    
    {"Features": "SNV + CNA + SV",
    "Model": "RandomForest",
    "Accuracy": multi_rf_acc,
    "Macro-F1": multi_rf_f1,},
])

print(results)


         Features               Model  Accuracy  Macro-F1
0        SNV-only  LogisticRegression   0.98750  0.987469
1  SNV + CNA + SV  LogisticRegression   0.99375  0.993746
2  SNV + CNA + SV        RandomForest   0.98125  0.981246


In [7]:

multi_omic['cancer_type']

0      Breast-AdenoCa
1       CNS-PiloAstro
2         Eso-AdenoCa
3          Kidney-RCC
4           Liver-HCC
            ...      
795        Kidney-RCC
796         Liver-HCC
797         Lymph-CLL
798    Panc-Endocrine
799     Prost-AdenoCA
Name: cancer_type, Length: 800, dtype: object

In [12]:
results.to_csv('../data/outputs/results.csv')

## Explainable AI

In [9]:
rf_importance = (pd.DataFrame({
        "feature": X_multi.columns,
        "importance": rf_multi.feature_importances_,}).sort_values("importance", ascending=False).reset_index(drop=True))

rf_importance # globally most influencial features from random forest

,feature,importance
0,sv_DUP,0.034995
1,snv_ms_SBS40,0.034014
2,snv_ms_SBS12,0.030731
3,snv_ms_SBS1,0.028147
4,sv_total,0.027412
...,...,...
230,snv_ms_driver_SPTAN1_Intron,0.000000
231,snv_ms_driver_ACO1_Intron,0.000000
232,snv_ms_driver_CAPN7_Intron,0.000000
233,snv_ms_driver_TRIO_Coding,0.000000


In [10]:
def lr_per_class_importance(lr_model, feature_names, top_k=10):
    """
    Return a dict: cancer_type -> DataFrame of top_k features with highest
    absolute weight for that class.
    """
    classes = lr_model.classes_ 
    coefs = lr_model.coef_

    per_class = {}

    for class_idx, class_label in enumerate(classes):
        weights = coefs[class_idx, :] # weights for this class vs all the others
        abs_weights = np.abs(weights)

        df = (pd.DataFrame({"feature": feature_names, "weight": weights, "abs_weight": abs_weights,})
            .sort_values("abs_weight", ascending=False)
            .reset_index(drop=True)
            .head(top_k))

        per_class[class_label] = df

    return per_class

# Pull top 10 featurs from 
per_class_multi = lr_per_class_importance(logreg_multi, feature_names = X_multi.columns, top_k=10)

# write out top drivers per cancer
for cancer, df in per_class_multi.items():
    df.to_csv(f'../data/explainable_ai/{cancer}_drivers.csv')
